In [1]:
from qiskit import Aer, QuantumCircuit, ClassicalRegister, QuantumRegister, IBMQ, execute
import csv
import numpy as np
import matplotlib.pyplot
from matplotlib.pyplot import *
from qiskit.tools.visualization import plot_histogram
from matplotlib.lines import Line2D


In [2]:
class CSVWriter:

    def __init__(self, QuantumCircuit, backends, input_bits, error_op, shots, filename):
        self.qc = QuantumCircuit
        self.error_op = error_op
        self.input_bits = input_bits
        self.jobs = []
        self.filename = filename
        self.__init_jobs(backends, shots)

    class __job:

        def __init__(self, SpecificBackend, qc, shots):
            self.qc = qc
            self.shots = shots
            provider = IBMQ.get_provider()
            self.backendname = SpecificBackend
            self.backend = provider.get_backend(SpecificBackend)
            self.result, self.id = self.__run()

        def __run(self):
            job = execute(self.qc, backend = self.backend, shots = self.shots)
            return job.result(), job.job_id()

    def __id(self, job):
        return job.id

    def __machine(self, job):
        return job.backendname

    def __shots(self, job):
        return job.shots

    def __KeyVals(self, job):
        dic = job.result.data()['counts']
        keys, vals = zip(*dic.items())
        return keys, vals

    def __init_jobs(self, backends, shots):
        print('Initializing Jobs...')
        for backend in backends:
            print('Running qc on '+backend+' for '+str(shots)+' shots...')
            self.jobs.append(self.__job(backend, self.qc, shots))
            print('done.')

    def __generate_labels(self, job):
        keys, vals = self.__KeyVals(job)
        labels = [['Machine'], ['input bits'], ['error operator'], ['no. of shots'], ['job id']]
        for key in keys:
            labels.append([key])
        return labels

    def __generate_data(self, job, Ldat):
        keys, vals = self.__KeyVals(job)
        Ldat[0].append(self.__machine(job))
        Ldat[1].append(self.input_bits)
        Ldat[2].append(self.error_op)
        Ldat[3].append(self.__shots(job))
        Ldat[4].append(self.__id(job))
        for i in range(len(keys)):
            Ldat[5+i].append(vals[i])
        
    def generate_csv(self):
        filename = self.filename
        labels = self.__generate_labels(self.jobs[0])
        for job in self.jobs:
            self.__generate_data(job, labels)
        with open(filename, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(labels)

In [2]:
#Loading IBMQ Account
IBMQ.save_account('0cc7f99e3937985e364e794988e2b600ad482f1ba5294581accde0d8909f2156270243a97a606fdd4331645411c18e424ccc17873383b572b175d894289a9804', overwrite=True)
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [6]:
# Then we construct our quantum circuit however we desire.
##Build the Circuit and Visualize   

##BASIC gate Decomposition with error operator W=H

t = np.arcsin(np.sqrt(1/3)) #t=theta/2
a = np.pi/4
b = np.pi-t

#Init Circuit
q = QuantumRegister(3)
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)

## encoding ##

qc.ry(-t,q[2]) 
qc.x(q[1])
qc.cx(q[1], q[2])
qc.u3(b,0,np.pi,q[2])
qc.cx(q[2],q[1])
qc.ry(a,q[1])
qc.cx(q[2],q[1])
qc.x(q[2])
qc.ry(-a,q[1])
qc.z(q[0])
qc.cx(q[0],q[1])
qc.cx(q[2],q[0])
qc.cx(q[1],q[2])
qc.x(q[1])

qc.barrier()

## error operators ##
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])

qc.barrier()

## decoding ##
qc.x(q[1])
qc.cx(q[1],q[2])
qc.cx(q[2],q[0])
qc.cx(q[0],q[1])
qc.z(q[0])
qc.ry(a,q[1])
qc.x(q[2])
qc.cx(q[2],q[1])
qc.ry(-a,q[1])
qc.cx(q[2],q[1])
qc.u3(b,0,np.pi,q[2])
qc.cx(q[1], q[2])
qc.x(q[1])
qc.ry(t,q[2])

##Prints the unitary matrix 

#unitary = Aer.get_backend('unitary_simulator')
#gate = execute(qc,unitary).result().get_unitary()
#print(gate)

## measurement ##
qc.barrier()
qc.measure(q[1],c[0])


#qc.draw()
CircFig=qc.draw(output="latex",filename='BasicCircFig.png')

In [5]:
# Next we establish a list of backends that we want to run on.  These must be strings.
backends = ['ibmq_valencia', 'ibmq_santiago', 'ibmq_vigo', 'ibmqx2', 'ibmq_ourense', 'ibmq_athens']
#Running the results and writing the info on a csv file
A = CSVWriter(qc, backends, '000', 'I', 2400, 'testBASIC.csv')
A.generate_csv()

Initializing Jobs...
Running qc on ibmq_valencia for 2400 shots...
done.
Running qc on ibmq_santiago for 2400 shots...
done.
Running qc on ibmq_vigo for 2400 shots...
done.
Running qc on ibmqx2 for 2400 shots...
done.
Running qc on ibmq_ourense for 2400 shots...
done.
Running qc on ibmq_athens for 2400 shots...
done.


In [4]:
#Run 1 Dec 13, 2020 
#bh_va={'0': 2027, '1': 373} 
#bh_sa={'0': 1830, '1': 570} 
#bh_vi={'0': 1976, '1': 424} 
#bh_yt={'0': 1910, '1': 490} 
#bh_ou={'0': 1922, '1': 478} 

#Run 2 Dec 15, 2020
#bh_va={'0': 1925, '1': 475} 
#bh_sa={'0': 1543, '1': 857} 
#bh_vi={'0': 1959, '1': 441} 
#bh_yt={'0': 1937, '1': 463} 
#bh_ou={'0': 2063, '1': 337} 
#bh_at={'0': 2069, '1': 331}

In [2]:
#Run 3 Dec 29, 2020
bh_va={'*0*': 1758, '*1*': 642} 
bh_sa={'*0*': 2085, '*1*': 315} 
bh_vi={'*0*': 2084, '*1*': 316} 
bh_yt={'*0*': 1933, '*1*': 467} 
bh_ou={'*0*': 2026, '*1*': 374} 
bh_at={'*0*': 2019, '*1*': 381}



In [3]:
colors= ['blue','orange','green', 'red','purple', 'cyan']

bhhist=plot_histogram([bh_va, bh_sa, bh_vi, bh_yt, bh_ou, bh_at], color=colors, title='Basic gates decomposition with W=H',figsize=(10,10))
yax = bhhist.axes[0]
yax.set_ylim(0,1)
ax = bhhist.gca() 


Legend_Lines = [Line2D([0], [0], color=colors[0], lw=4),
                Line2D([0], [0], color=colors[1], lw=4),
                Line2D([0], [0], color=colors[2], lw=4),
                Line2D([0], [0], color=colors[3], lw=4),
                Line2D([0], [0], color=colors[4], lw=4),
                Line2D([0], [0], color=colors[5], lw=4)]

bhhist.legend(Legend_Lines, ['ibmq_valencia', 'ibmq_santiago', 'ibmq_vigo', 'ibmqx2', 'ibmq_ourense', 'ibmq_athens'], bbox_to_anchor=(0.95, 0.95), bbox_transform=ax.transAxes)

bhhist.savefig('3BH.png')
